In [2]:
from uncertainties import measurement as ms
import pandas as pd

data = pd.read_csv('data.csv')
k = ms.Observable(symbol='k', definition='m*(T - T_e)/(T_e - T_0) - M', data=data, last_row_error=True)

TypeError: loop of ufunc does not support argument 0 of type Float which has no callable sqrt method

In [6]:
takes_errors = []
partial_sq_errors_ = []
for take in range(len(k.data)):
    take_value = k.definition.subs(k.data.iloc[take].items())
    alpha_args = {}
    for symbol in k.data.columns:
        alpha_args[symbol] = k.data.iloc[take].copy()
        alpha_args[symbol].loc[symbol] = alpha_args[symbol].loc[symbol] + k.vars_errors.loc[symbol]
    partial_sq_errors = array([abs(k.definition.subs(arg[1].items()) - take_value)**2 for arg in alpha_args.items()])
    partial_sq_errors_.append(partial_sq_errors)
    take_error = partial_sq_errors.sum()
    takes_errors.append(take_error)


KeyError: 'error'

In [24]:
import sympy as sym
partial_derivatives = {var.name : sym.diff(k.definition, var) for var in k.definition.free_symbols}

In [25]:
take = 1
for var in k.definition.free_symbols:
    print(var.name + ':', (partial_derivatives[var.name].subs(k.data.iloc[take].items())**2)*k.vars_errors[var.name])


M: 0.100000000000000
m: 0.250694444444444
T_0: 20.9853011670525
T: 8.37086805555556
T_e: 55.8639180652006


In [39]:
import numpy as np
a = np.array([
    (partial_derivatives[var.name].subs(k.data.iloc[take].items())**2) * k.vars_errors[var.name] for var in k.definition.free_symbols]).sum()
sym.sqrt(a)

9.25044765037093

9.25044765037093